In [10]:
#SCRAPING ENVOIREMENT

In [8]:
#Importing libaries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import csv

## De følgende celler skal IKKE køres

In [447]:
#Importing page-links
"""
list_url = []
for page in range(1,201,1):
    url = f'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page={page}'
    list_url.append(url)

print(len(list_url))
"""




200


In [449]:
#Retriving HTMLS
"""
list_htmls =[]
for url in list_url:
    response = requests.get(url, headers={'name': 'Mathias Rasmussen', 'email': 'kbd655@alumni.ku.dk'})
    list_htmls.append(response.text)
    time.sleep(0.5)

"""
    

In [451]:
# Finding links on appartments
"""
links = []
for html in list_htmls:
    soup = BeautifulSoup(html, 'lxml')
    pattern = re.compile(r'href\":\"(/addresses/[^\"]+)')
    matches = pattern.findall(str(soup))

    apartments_list = list(set(matches))


    base_url = "https://www.boligsiden.dk"
    apartment_links = [base_url + link for link in apartments_list] #This list contains appartments
    links.append(apartment_links)

flattened_links = list(itertools.chain(*links))
print(len(flattened_links)) #Counting number of links
"""

9996


In [453]:
#Saving list of links to be scrapped
"""
with open('rawdata/links.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for link in flattened_links:
        writer.writerow([link])
"""


# HER STARTER SCRAPING PROCESSEN

In [10]:
#Importing relevant list of links:
flattened_links = []

with open('rawdata/links.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        flattened_links.append(row[0])  # Assuming each link is in the first column

print(len(flattened_links))

9996


In [12]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options=op)

In [30]:
#Carefull! Runs with X-links at a time
#NANNAS VERSION
dict = {}
links_med_fejl = []

def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
            
        #Finder label hvor afstand er under
        """
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        """
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        #################   NY KODE med skoler    ########################
        #Skolenavn
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1



        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[5000:6000]), total=len(flattened_links[5000:6000])))


 10%|███▉                                     | 96/1000 [02:11<22:40,  1.50s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50ac-7c2f-32b8-e044-0003ba298018 - list index out of range


 25%|█████████▉                              | 247/1000 [06:59<29:53,  2.38s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50a5-be5c-32b8-e044-0003ba298018 - list index out of range


 39%|██████████████▋                       | 386/1000 [10:13<3:02:37, 17.85s/it]

...

 86%|██████████████████████████████████▏     | 855/1000 [26:14<37:49, 15.65s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b2-3cc2-32b8-e044-0003ba298018 - list index out of range


 88%|███████████████████████████████████     | 876/1000 [26:46<07:18,  3.54s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b3-6688-32b8-e044-0003ba298018 - list index out of range


100%|███████████████████████████████████████| 1000/1000 [28:33<00:00,  1.71s/it]


In [2]:
#Sikkerhedskopi
"""
#Carefull! Runs with X-links at a time
dict = {}
links_med_fejl = []

def fetch_data(link):
    #service = Service(ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=service)
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
        #Finder label hvor afstand er under
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'Kommune_tekst': kommune_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'Kommune_tekst': kommune_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[0:50]), total=len(flattened_links)))
"""

NameError: name 'ThreadPoolExecutor' is not defined

In [32]:
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
    
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        
        #################   NY KODE med skoler    ########################
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))


100%|███████████████████████████████████████████| 10/10 [02:08<00:00, 12.84s/it]


In [376]:
#sikkerheds

"""
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
        #Finder label hvor afstand er under
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))
""""

NameError: name 'liste_med_fejl' is not defined

## Lav først til DataFrame og efterfølgende til CSV fil

In [34]:
#Converting dict to dataframe:
DF_NAME = 'index_5000_6000' #Her kalder du din Dataframe/CSV-fil noget unikt.

df_DF_NAME = pd.DataFrame.from_dict(dict, orient='index')
pd.set_option('display.max_colwidth', None)
df_DF_NAME



,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Elevator,Rooms,Build,Toilets,Altan,Terasse,Postal code,link,Grund,Toilet
"Ryetvej 59, 1. tv.,3500",2.295.000 kr.,Lille Værløse Skole,323 m,7.6,3.8,Ejerlejlighed,Energimærke C,Mellem,"24,88%","21,5‰.",...,Elevator: Nej,3 værelser,Opført 1968,1 toilet,Altan: Ja,Terrasse: Nej,3500,https://www.boligsiden.dk/addresses/0a3f50a7-a5c9-32b8-e044-0003ba298018,NaN,NaN
"Flagbakkevej 26, st. 37.,9990",2.195.000 kr.,Skagen Skole,"3,27 km",Ikke oplyst,Ikke oplyst,Ejerlejlighed/Fritidsbolig,Energimærke E,Meget lav,"26,2%","29,9‰.",...,Elevator: Nej,2 værelser,Opført 2007,-,Altan: Nej,Terrasse: Nej,9990,https://www.boligsiden.dk/addresses/5338c3f8-bb0d-00e2-e044-0003ba298018,NaN,NaN
"Nørregårdsvej 10,5771",1.895.000 kr.,Issø-skolen,"2,75 km",6.8,3.7,Villa,Energimærke B,Meget høj,"26,3%","23,32‰.",...,NaN,6 værelser,Opført 1906,1 toilet,NaN,NaN,5771,https://www.boligsiden.dk/addresses/0a3f50b2-7d7e-32b8-e044-0003ba298018,Grund: 1382 m²,NaN
"Østre Ringvej 17,8305",1.245.000 kr.,Samsø Skole,"14,62 km",6.7,3.6,Villa/Fritidsbolig,Energimærke E,Høj,"25,9%","30,75‰.",...,NaN,3 værelser,Opført 1894,1 toilet,NaN,NaN,8305,https://www.boligsiden.dk/addresses/0a3f50c1-b251-32b8-e044-0003ba298018,Grund: 144 m²,NaN
"Langemarksvej 10,7100",1.195.000 kr.,Lindved Skole,455 m,6.9,3.6,Villa,Energimærke D,Høj,"25,52%",18‰.,...,NaN,6 værelser,Opført 1976,2 toiletter,NaN,NaN,7100,https://www.boligsiden.dk/addresses/0a3f50bc-7fbe-32b8-e044-0003ba298018,Grund: 909 m²,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Rugmarksvej 19,5800",2.295.000 kr.,Danehofskolen,"1,29 km",6.8,3.7,Villa,Energimærke B,Høj,"26,3%","27,83‰.",...,NaN,5 værelser,Opført 1969,1 toilet,NaN,NaN,5800,https://www.boligsiden.dk/addresses/0a3f50b3-6688-32b8-e044-0003ba298018,Grund: 913 m²,NaN
"Jægergårdsgade 46, 1.,8000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,fejl,fejl,https://www.boligsiden.dk/addresses/0a3f50c3-5434-32b8-e044-0003ba298018,NaN,fejl
"Lotusvej 5,4030",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,fejl,fejl,https://www.boligsiden.dk/addresses/0a3f50ab-0a63-32b8-e044-0003ba298018,NaN,fejl
"Svinget 3,4300",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,fejl,fejl,https://www.boligsiden.dk/addresses/0a3f50ad-6be4-32b8-e044-0003ba298018,NaN,fejl


In [36]:
#Converts to csv-file and save in "Rawdata"
df_DF_NAME.to_csv(f'rawdata/{DF_NAME}.csv', index=True)

